In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np

C:\Users\fenix\AppData\Local\Temp/ipykernel_17304/3926170050.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\fenix\AppData\Local\Temp/ipykernel_17304/3926170050.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
missing_value_formats = ["n.a.","?","NA",":", "na", "--"]
data = pd.read_csv("apro_mk_colm_1_Data.csv",na_values = missing_value_formats)

In [3]:
data["DAIRYPROD"]=data["DAIRYPROD"].str.replace("[^\w\s]","")
data['TIME'] = data['TIME'].replace({'M':'-'}, regex=True)
data = data.drop(["Flag and Footnotes"], axis=1)


C:\Users\fenix\AppData\Local\Temp/ipykernel_17304/4140797261.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data["DAIRYPROD"]=data["DAIRYPROD"].str.replace("[^\w\s]","")


In [4]:
milk_delivered_to_dairies=data.query("DAIRYPROD=='Raw cows milk delivered to dairies'")
milk_delivered_to_dairies = milk_delivered_to_dairies.drop(["DAIRYPROD"], axis=1)

milk_delivered_to_dairies_tonnes=milk_delivered_to_dairies.query("UNIT=='Thousand tonnes'")
milk_delivered_to_dairies_Fat=milk_delivered_to_dairies.query("UNIT=='Fat content (% of product weight)'")
milk_delivered_to_dairies_Protein=milk_delivered_to_dairies.query("UNIT=='Protein content (% of product weight)'")


milk_delivered_to_dairies_tonnes = milk_delivered_to_dairies_tonnes.rename(columns={"Value": "Thousand_tonnes_to_dairies"})
milk_delivered_to_dairies_Fat=milk_delivered_to_dairies_Fat.rename(columns={"Value": "Fat_content"})
milk_delivered_to_dairies_Protein=milk_delivered_to_dairies_Protein.rename(columns={"Value": "Protein_content"})




milk_delivered_to_dairies_tonnes = milk_delivered_to_dairies_tonnes.drop(["UNIT"], axis=1)
milk_delivered_to_dairies_Fat = milk_delivered_to_dairies_Fat.drop(["UNIT"], axis=1)
milk_delivered_to_dairies_Protein = milk_delivered_to_dairies_Protein.drop(["UNIT"], axis=1)


milk_delivered_to_dairies=pd.merge(milk_delivered_to_dairies_tonnes , milk_delivered_to_dairies_Fat ,on=["TIME","GEO"]) #INNER MERGE
milk_delivered_to_dairies=pd.merge(milk_delivered_to_dairies , milk_delivered_to_dairies_Protein, on=["TIME","GEO"])
data=pd.merge(data , milk_delivered_to_dairies, on=["TIME","GEO"])


In [5]:
data=data.dropna(subset=["Value"])
data=data.drop("Thousand_tonnes_to_dairies",axis=1)
indexNames = data[ (data['UNIT'] == "Fat content (% of product weight)")
                | (data['UNIT'] == "Protein content (% of product weight)") ].index
data.drop(indexNames , inplace=True)
#print(data)

In [6]:
data["Date"] = data["TIME"] + "-01"
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")
data = data.rename(columns={"GEO": "region", "DAIRYPROD": "type" })


In [7]:
data.head()

,TIME,region,type,UNIT,Value,Fat_content,Protein_content,Date
60,2000-01,European Union - 15 countries (1995-2004),Raw cows milk delivered to dairies,Thousand tonnes,9367.34,4.20,3.32,2000-01-01
75,2000-01,Belgium,Raw cows milk delivered to dairies,Thousand tonnes,260.02,4.18,3.34,2000-01-01
81,2000-01,Belgium,Drinking milk,Thousand tonnes,53.93,4.18,3.34,2000-01-01
84,2000-01,Belgium,Cream for direct consumption,Thousand tonnes,5.50,4.18,3.34,2000-01-01
87,2000-01,Belgium,Milk and cream powders excluding skimmed milk ...,Thousand tonnes,6.99,4.18,3.34,2000-01-01


In [9]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
from dash.dependencies import Output, Input


external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
        "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Milk Analytics: Understand production of milk!"

app.layout = html.Div(
    children=[
        html.Div(
            children=[
                html.P(children="🐄", className="header-emoji"),
                html.H1(
                    children="Milk Analytics", className="header-title"
                ),
                html.P(
                    children="Analyze the behavior of production milk"
            " and the number of solids content in the EU"
            " between 2000 and 2022",
                    className="header-description",
                ),
            ],
            className="header",
        ),
        html.Div(
            children=[
                html.Div(
                    children=[
                        html.Div(children="Region", className="menu-title"),
                        dcc.Dropdown(
                            id="region-filter",
                            options=[
                                {"label": region, "value": region}
                                for region in np.sort(data.region.unique())
                            ],
                            value="Ireland",
                            clearable=False,
                            className="dropdown",
                        ),
                    ]
                ),
                html.Div(
                    children=[
                        html.Div(children="Type", className="menu-title"),
                        dcc.Dropdown(
                            id="type-filter",
                            options=[
                                {"label": Dairy_product, "value": Dairy_product}
                                for Dairy_product in data.type.unique()
                            ],
                            value="Raw cows milk delivered to dairies",
                            clearable=False,
                            searchable=False,
                            className="dropdown",
                        ),
                    ],
                ),
                html.Div(
                    children=[
                        html.Div(
                            children="Date Range",
                            className="menu-title"
                            ),
                        dcc.DatePickerRange(
                            id="date-range",
                            min_date_allowed=data.Date.min().date(),
                            max_date_allowed=data.Date.max().date(),
                            start_date=data.Date.min().date(),
                            end_date=data.Date.max().date(),
                        ),
                    ]
                ),
            ],
            className="menu",
        ),
        html.Div(
            children=[
                html.Div(
                    children=dcc.Graph(
                        id="production-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="fat-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
                html.Div(
                    children=dcc.Graph(
                        id="Protein-chart", config={"displayModeBar": False},
                    ),
                    className="card",
                ),
            ],
            className="wrapper",
        ),
    ]
)


@app.callback(
    [Output("production-chart", "figure"), Output("fat-chart", "figure"),Output("Protein-chart", "figure")],
    [
        Input("region-filter", "value"),
        Input("type-filter", "value"),
        Input("date-range", "start_date"),
        Input("date-range", "end_date"),
    ],
)
def update_charts(region, milk_type, start_date, end_date):
    mask = (
        (data.region == region)
        & (data.type == milk_type)
        & (data.Date >= start_date)
        & (data.Date <= end_date)
    )
    filtered_data = data.loc[mask, :]
    production_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Value"],
                "type": "lines",
                #"hovertemplate": "$%{y:.2f}<extra></extra>",
            },
        ],
        "layout": {
            "title": {
                "text": "PRODUCTION",
                "x": 0.05,
                "xanchor": "left",
            },
            "xaxis": {"fixedrange": True},
            #"yaxis": {"tickprefix": "$", "fixedrange": True},
            "colorway": ["#17B897"],
        },
    }

    fat_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Fat_content"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Fat_content", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#E12D39"],
        },
    }
    
    Protein_chart_figure = {
        "data": [
            {
                "x": filtered_data["Date"],
                "y": filtered_data["Protein_content"],
                "type": "lines",
            },
        ],
        "layout": {
            "title": {"text": "Protein_content", "x": 0.05, "xanchor": "left"},
            "xaxis": {"fixedrange": True},
            "yaxis": {"fixedrange": True},
            "colorway": ["#2d39e1"],
        },
    }    
    
    return production_chart_figure, fat_chart_figure,Protein_chart_figure


if __name__ == '__main__':
    app.run_server(host='localhost',port=8005)

Dash is running on http://localhost:8005/

Dash is running on http://localhost:8005/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8005/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2022 10:52:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [20/May/2022 10:52:07] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
